# JSSP comparação entre euristicas Classicas: codigo fonte

## importação de libs

In [ ]:
# %pip install mealpy
# %pip install numpy
%pip install -r ../requirements.txt


In [ ]:
import os
import numpy as np
import mealpy
import importlib.util
from classes.jssp import jssp
from classes.job import Jssp_job


## funções adicionais

In [ ]:
# importar casos de teste
def import_tests_cases(nome_dict : str) -> dict:

    caminho_absoluto = os.path.abspath("../tests/test1.py")

    spec = importlib.util.spec_from_file_location("modulo_temp", caminho_absoluto)
    
    if spec is None or spec.loader is None:
        raise ImportError(f"Não foi possível carregar o módulo do arquivo: {caminho_absoluto}")
    
    modulo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(modulo)

    return getattr(modulo, nome_dict)



In [ ]:
# apenas testanto importação
test = import_tests_cases("test")
print(type(test))
print(test)

<class 'dict'>
{'jobs': {'job_1': [([1], [], 1)], 'job_2': [([2], [], 1)]}, 'machine_downtimes': {}, 'timespan': 2}


## testes

### Simulated annealing

In [9]:
data = import_tests_cases("test")


AttributeError: module 'modulo_temp' has no attribute 'data'